In [1]:
import numpy
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_text as text
import tensorflow_hub

In [5]:
def clean_df(df):
    df = df.drop(['q_title', 'q_id', 'line_id'], axis=1)
    return df

train_src = "./codenrock_ds/TextGenerationDetection/train.csv"
test_src = "./codenrock_ds/TextGenerationDetection/test.csv"
df = pd.read_csv(train_src)
testing_df = pd.read_csv(test_src)

df = clean_df(df)
testing_df = clean_df(testing_df)
train, valid, test = numpy.split(df.sample(frac=1), [int(0.6 * len(df)), int(0.8 * len(df))])
train.head()

d:\progdata\anaconda\envs\ml\lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


,label,ans_text
4043,hu_answer,We'll combine graph-theory and probability:\nO...
2440,ai_answer,"Ну что ж, молодежь сегодня такая духовная! Есл..."
3026,hu_answer,"Начнём с шутки:\n""Он пришёл на свидание с саже..."
867,hu_answer,That's because you use the .NET 4.0 version of...
3083,hu_answer,В одной хорошей книге встретила такое выражени...


In [6]:
def scale_dataset(data):
    data["label"] = (data["label"] == "hu_answer").astype(int)
    y = data[data.columns[0]].values
    X = data.drop(data.columns[0], axis=1).values
    data = numpy.hstack((X, numpy.reshape(y, (-1, 1))))
    
    return data, X, y

In [7]:
train_data, X_train, y_train = scale_dataset(train)
valid_data, X_valid, y_valid = scale_dataset(valid)
test_data, X_test, y_test = scale_dataset(train)

In [8]:
bert_preprocess = tensorflow_hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = tensorflow_hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [6]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

layer = tf.keras.layers.Dropout(0.1, name='dropout')(outputs['pooled_output'])
layer = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(layer)

model = tf.keras.Model(inputs=[text_input], outputs=[layer])


In [7]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics='accuracy'
)

In [ ]:
model.fit(X_train, y_train, epochs=12)
val_loss = model.evaluate(X_valid, y_valid)[0]

In [2]:
model = tf.keras.models.load_model('newest_model')

In [16]:
valid_data

array([['The error code 215 in Twitter API indicates that the authentication data provided is invalid or expired. This error can occur due to various reasons, such as incorrect consumer key or secret, access token or secret, or expired access token. To resolve this issue, you need to ensure that the authentication data provided is correct and up-to-date. You can regenerate the access token and secret from the Twitter developer dashboard and update them in your application. Additionally, make sure that the time and date on your device are correct, as an incorrect time can also cause authentication errors.',
        0],
       ['A modal view will cover the view it is pushed on top of as well as the navigation bar for your navigation controller. However, if you use the -presentModalViewController:animated: approach, then once the animation finishes the view just covered will actually disappear, which makes any transparency of your modal view pointless. (You can verify this by implementing

In [22]:
yet_another_q = model.predict(["я нейронная сеть"])
print("human" if yet_another_q >= 0.5 else "ai")

1/1 [==============================] - 1s 687ms/step
human


In [9]:
from sklearn.metrics import classification_report
y_train_predict = model.predict(X_train)

91/91 [==============================] - 948s 10s/step


In [14]:
y_binary = numpy.where(y_train_predict > 0.5, 1, 0)
report = classification_report(y_binary, y_train)
print(report)

              precision    recall  f1-score   support

           0       0.66      0.81      0.72      1181
           1       0.84      0.71      0.77      1710

    accuracy                           0.75      2891
   macro avg       0.75      0.76      0.75      2891
weighted avg       0.77      0.75      0.75      2891



In [15]:
y_check = model.predict(X_valid)

31/31 [==============================] - 338s 11s/step


In [16]:
y_check_binary = numpy.where(y_check >= 0.5, 1, 0)
valid_report = classification_report(y_check_binary, y_valid)
print(valid_report)